In [ ]:
# import gzip
# import json

# def read_jsonl_gz(file_path):
#     with gzip.open(file_path, 'rt') as f:
#         for line in f:
#             yield json.loads(line)

# file_path = './data/manifests/genshin_supervisions_dev.jsonl.gz'

# # 逐行读取和解析 JSON 数据
# json_data = list(read_jsonl_gz(file_path))

# # 查看数据
# json_data


In [1]:
import os
from bin.myinfer import InferModel

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
model = InferModel(checkpoint="exp/valle2/best-valid-loss.pt")
# model512 = InferModel(checkpoint='exp512/valle1/best-valid-loss.pt',decoder_dim=512)
# model256 = InferModel(checkpoint='exp256/valle/best-valid-loss.pt',decoder_dim=256,nhead=8,num_decoder_layers=6)

/home/ldy/anaconda3/envs/valle/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import json
from typing import List
from collections import OrderedDict
from os import PathLike
from pathlib import Path

import sys

# 清除导入的缓存
if "bin.utils" in sys.modules:
    del sys.modules["bin.utils"]

from bin.utils import *


class GenShinDataWrapper:
    def __init__(
        self,
    ) -> None:
        dir = Path("download/genshin")
        self.dir = dir
        self.voice = json.load(open(dir / "result_chs.json"))
        self.voice_test = json.load(open(dir / "result_chs_test.json"))

        # 按照 value['npcName'] 进行分组
        grouped_dict = OrderedDict()
        for key, value in self.voice.items():
            group_key = value["npcName"]
            grouped_dict.setdefault(group_key, []).append(key)
        self.grouped_dict = grouped_dict
        self.text_processor = TextProcessor()

    def get_text_audio(self, id):
        value = self.voice[id]
        return value["text"], self.dir / value["fileName"]

    def create_file(
        self,
        file_name: PathLike,
        ids: List[str],
        texts: List[str],
        infer_dir: PathLike = "audios/genshin",
        copy: bool = False,
    ):
        texts = [self.text_processor.process_str(t) for t in texts]
        return create_file_base(
            file_name, ids, self.get_text_audio, texts, infer_dir, copy
        )

    def get_npc_ids(
        self, name: str, count: int = 5, min_len: int = 8, max_len: int = 18
    ):
        ids = [
            key
            for key in self.grouped_dict[name]
            if min_len <= len(self.voice[key]["text"]) <= max_len
        ]
        return random_samples(ids, count)

    def get_test_ids(self, count: int = 5, min_len=8, max_len=18) -> List[str]:
        ids = [
            key
            for key, value in self.voice_test.items()
            if min_len <= len(value["text"]) <= max_len
        ]
        return random_samples(ids, count)

    def create_train_demo(
        self, file_name="genshin.txt", out_name="../../../genshin.txt"
    ):
        with open(file_name) as f1, open(out_name,'w') as f2:
            for line in f1.readlines():
                if line:
                    f2.write(copy_file_in_line(line, "audios/genshin_train"))

    def nice_write_line(
        self, file_name: PathLike = "genshin.txt", idxs: List[int] = None
    ):
        lines = [l.strip() for l in open(file_name).readlines() if l.strip()]
        if not idxs:
            return
        target_path = "../../../genshin_nice.txt"
        with open(target_path, "a") as f:
            for idx in idxs:
                line = copy_file_in_line(
                    lines[idx], target_path="audios/aishell2_nice"
                )
                f.write(line + "\n")
                print(f"add_line: {line}")

In [11]:
wrapper = GenShinDataWrapper()

# ids = ["c15260d94fa350c1"] * 5
# ids = wrapper.get_test_ids(5)

ids = wrapper.get_npc_ids("派蒙",count=3,min_len=10) \
+ wrapper.get_npc_ids("胡桃",count=3,min_len=10) \
+ wrapper.get_npc_ids("行秋",count=3,min_len=10) \
+ wrapper.get_npc_ids("艾尔海森",count=3,min_len=10)

texts = get_tts_texts("mid",count=12)

ids = wrapper.get_npc_ids("派蒙",min_len=10)
texts = get_tts_texts('long')

file_name = wrapper.create_file("genshin.txt", ids, texts, "audios/genshin")

那万一她用不着你帮忙,你怎么办?	download/genshin/Chinese/VO_AQ/VO_paimon/vo_CYAQ001_1_paimon_25b.wav	在广阔的天空中,勇敢地展翅高飞,去征服属于自己的蓝天.风儿在耳边低语,让心灵自由翱翔.感受飞行的自由与快乐,看着身下的世界渐行渐远,那种豪情壮志令人心潮澎湃.	audios/genshin/273446959cf8f403_infer.wav
咦,你已经不会再难受了吗?真的吗?	download/genshin/Chinese/VO_AQ/VO_paimon/vo_XMAQ003_12_paimon_07.wav	音乐是心灵的语言,它能穿透时空的界限,触动最柔软的内心深处.沉浸在音乐中,身心得到疗愈与抚慰,情绪得到释放与宣泄.音符跳跃在空气中,带着节奏和情感,将我们带入一个全新的境界,与自我和谐共舞.	audios/genshin/482d34596edaf7ae_infer.wav
五十年?是什么重要的日子吗?	download/genshin/Chinese/VO_LQ/VO_paimon/vo_XGLQ003_3_paimon_01.wav	相信自己,你是独一无二的存在.你拥有无限的潜力和能力,可以超越自我,创造出令人惊艳的奇迹.不要害怕追逐梦想,不要停止向前,因为你拥有改变世界的力量,只需要勇敢地展现真实的自己,去创造你独特的轨迹.	audios/genshin/fbe427e68489f832_infer.wav



In [85]:
wrapper = GenShinDataWrapper()

wrapper.nice_write_line('genshin.txt',idxs=[2,4])

成功将文件 'download/genshin/Chinese/VO_LQ/VO_npc/vo_XHLQ002_5_izumiNaki_03.wav' 复制到 'audios/genshin_nice/vo_XHLQ002_5_izumiNaki_03.wav'。
成功将文件 'audios/genshin/c68ab184654664be_infer.wav' 复制到 'audios/genshin_nice/c68ab184654664be_infer.wav'。
add_line: 真的吗?不愧是珊瑚宫大人,太厉害了!	audios/genshin_nice/vo_XHLQ002_5_izumiNaki_03.wav	成功并非是一蹴而就的,它需要付出汗水与努力,在坚持不懈的追求中,我们才能看到成功的曙光.	audios/genshin/c68ab184654664be_infer.wav
成功将文件 'download/genshin/Chinese/VO_LQ/VO_npc/vo_NHDLQ002_9_basimah_02.wav' 复制到 'audios/genshin_nice/vo_NHDLQ002_9_basimah_02.wav'。
成功将文件 'audios/genshin/c9107927aa8fa1fd_infer.wav' 复制到 'audios/genshin_nice/c9107927aa8fa1fd_infer.wav'。
add_line: 喂喂,说什么呢,黛比.	audios/genshin_nice/vo_NHDLQ002_9_basimah_02.wav	在人类文明的长河中,无数英勇的先驱者翻山越岭,探索未知,用智慧和勇气书写着壮丽的篇章,将世界推向更加辉煌的未来.	audios/genshin/c9107927aa8fa1fd_infer.wav


In [ ]:
# model256.infer_by_file(file_name)
# model512.infer_by_file(file_name)
model.infer_by_file(file_name)

show_audios(file_name)

In [ ]:
wrapper.create_train_demo()

In [ ]:
file_name = "../../../test1.txt"
ids = ["01", "02", "03", "04", "05", "06", "07", "08"]
text_prompts = [
    "经过屏幕上每一颗像素",
    "光线跟踪是一种真实的显示物体的方法",
    "特別的美",
    "去看了一下我們的牡丹",
    "你爱我们的家吗",
    "你也会爱我们的家的啊",
    "我来到这个世界上",
    "妈妈给我说过",
]
text_processor = TextProcessor()

create_file_base(
    file_name,
    ids=ids,
    get_info=lambda id: (text_prompts[int(id)-1],f'audios/test1/{id}.wav'),
    texts=[text_processor.process_str(t) for t in get_tts_texts("mid",count=10)],
    infer_dir="audios/test1",
)

model.infer_by_file(file_name)

show_audios(file_name)

In [ ]:
from bin.myinfer import InferModel

model_empty = InferModel("")

with open(file_name) as file:
    lines = [l.strip() for l in file.readlines()]
    for line in lines:
        if not line:
            break
        sp = line.split("\t")
        model_empty.audio_codec_test(sp[1], sp[3])

show_audios(file_name)

In [8]:
import re

text = '㶧'

# 使用正则表达式匹配中文字符
pattern = re.compile(r'[\u4e00-\u9fff]+')
matches = re.findall(pattern, text)

print(matches)  # 输出：['你好', '世界']


[]
